In [1]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold


In [2]:
train = pd.read_csv("../input/train_order_info.csv")
test = pd.read_csv("../input/test_order_info.csv")
train.shape,test.shape

((2415645, 10), (737100, 10))

In [4]:
print(train.columns)
train.head()

Index(['amt_order', 'type_pay', 'time_order', 'sts_order', 'phone',
       'unit_price', 'id', 'no_order_md5', 'name_rec_md5', 'product_id_md5'],
      dtype='object')


,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5
0,39.0,在线支付,2017-04-15 09:21:18,等待收货,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN
1,39.0,在线支付,2017-04-15 09:21:18,完成,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN
2,288.0,在线支付,2016-10-24 08:57:06,NaN,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN
3,428.0,在线支付,2016-10-27 20:04:48,完成,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN
4,428.0,在线支付,2016-10-27 20:04:48,NaN,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN


In [3]:
fulldata = pd.concat([train,test],axis = 0,ignore_index = False)
fulldata.isnull().sum().sort_values(ascending = False)/fulldata.shape[0]
# c = fulldata.sts_order.sort_values(ascending = False).index[0]
# fulldata.sts_order.fillna(c,inplace = True)

product_id_md5    0.735046
unit_price        0.735037
sts_order         0.123957
name_rec_md5      0.080888
type_pay          0.072888
phone             0.055974
amt_order         0.030502
no_order_md5      0.025304
time_order        0.025304
id                0.000000
dtype: float64

In [14]:
mm = train.ix[train.sts_order == "完成"]
len(mm.id.unique())

54345

# 下面开始处理数据

In [4]:
#购物有京东，线下购物，彩票 等，如何细分呢   ？

'''
完成               2147487已取消               367100
充值成功              338095未抢中               120283
已完成                64176订单取消               60717
等待收货               21889充值失败;退款成功           8164
出票成功                6985退款完成                3950
等待付款                2453已收货                 1926
充值失败                1554正在出库                1171
出票失败                1106退款成功                 935
商品出库                 917已晒单                  793
抢票已取消                689正在处理                 564
等待审核                 282付款成功                 215
预订结束                 194购买成功                 187
失败退款                 179请上门自提                 86
订单已取消                 74等待处理                  67
配送退货                  66已退款                   64
预约完成                  49等待退款                  44
缴费成功                  44等待付款确认                34
未入住                   30下单失败                  30
已取消订单                 27已确认                   23
过期关闭                  23正在送货（暂不能上门自提）         17
已入住                   11充值失败;退款处理中             9
正在充值                   9商品退库                   5
等待分期付款                 4支付失败                   4
部分充值成功;退款成功            3过期放弃                   3
等待揭晓                   2等待厂商处理                 2
预订中                    1等待发码                   1
发货中                    1  退款中                    1
Name: sts_order, dtype: int64


在线支付          2451964货到付款           297918在线             134974
混合支付            19506分期付款             6733在线+京豆            3247
上门自提             1861在线+限品东券           892余额                727
京豆                689邮局汇款              669在线+东券支付           580
公司转账              565京豆混合支付            516前台自付              496
京豆支付              424在线+定向东券           254在线+全品东券           177
定向东券              162在线+东券             143白条支付              104
东券混合支付             78在线预付               57京豆东券混合支付           48
在线支付               38定向京券支付             36积分支付               18
定向京券               18在线+余额              16京券混合支付              9
限品京券                6余额+限品东券             5在线+京券支付             5
全品京券                3在线+全品京券             2在线+余额+限品东券          1
在线+限品京券             1分期付款(招行)            1高校代理-自己支付           1
京券全额支付              1京豆京券混合支付            1高校代理-代理支付           1
京券支付                1
Name: type_pay, dtype: int64

处理前：1.中间状态全部转换为最终状态；2.一个订单号的条目（已完成全部合并为一个数目）

统计有几种支付方式，常见的支付方式是什么，（混合、东券、京劵、京豆、余额（这一个条目代表网上购物活跃度））多少次，
白条、分期 有多少次

订单状态+支付方式  保留原来的状态（用于统计多少种方式），并添加一列 自己转换的1列（数字合并），
'''
def findname(x): 
    if x.find(u"混合") !=-1: 
        return 1
    if x.find(u"京豆") !=-1: 
        return 1
    if x.find(u"京劵") !=-1: 
        return 1
    if x.find(u"东券") !=-1: 
        return 1
    if x.find(u"余额") !=-1: 
        return 1
    else:
        return 0

def tranf(x):
    x = str(x)
    if x.startswith(('完成','充值成功','已完成','出票成功','已收货','已晒单',"过期关闭"
                    '付款成功','购买成功','缴费成功','已确认','已入住','过期放弃',
                    '部分充值成功;退款成功','未入住','未抢中','预约完成')):
         x = 1
    elif x.startswith((                
                     '等待收货','等待付款','发货中',
                     '预订中','等待分期付款','正在充值',
                       '正在送货（暂不能上门自提）','等待付款确认',
                     '正在出库','等待厂商处理','等待发码','等待揭晓',
                      '请上门自提','商品出库')):    
        x = 2
    
    else:
        x = 3   
    return x

In [5]:
train = pd.read_csv("../input/train_order_info.csv")
test = pd.read_csv("../input/test_order_info.csv")
fulldata = pd.concat([train,test],axis = 0,ignore_index = True)
train.head(),train.shape,test.shape
# fulldata = fulldata[0:300]

#先填充缺失值
cate_feature = ['type_pay', 'phone',
        'no_order_md5', 'name_rec_md5']
for i in range(len(cate_feature)):
#     print(fulldata[cate_feature[i]].value_counts().sort_values(ascending = False).index[0])
    fulldata[cate_feature[i]].fillna(fulldata[cate_feature[i]].value_counts().sort_values(ascending = False).index[0],inplace = True)


fulldata['amt_order'].fillna(fulldata['amt_order'].median(),inplace = True)
fulldata['amt_order'].value_counts().sort_values(ascending = False)
ulimit = np.percentile(fulldata['amt_order'],99)
fulldata['amt_order'].loc[fulldata['amt_order']>ulimit] = ulimit
fulldata['amt_order'] = fulldata['amt_order'].apply(lambda x: abs(x))   
np.min(fulldata['amt_order']),np.max(fulldata['amt_order'])
fulldata["lossNUm"] = fulldata.isnull().sum(axis = 1)
#先把状态填充，找出有多少状态取消，多少状态
fulldata["sts_order_number"] = 0
fulldata["type_pay_special"] = 0
fulldata["sts_order"].fillna(fulldata["sts_order"].value_counts().sort_values(ascending = False).index[0],inplace = True)
fulldata["sts_order_number"] = fulldata["sts_order"].apply(lambda x:tranf(x))
fulldata["type_pay"] = fulldata["type_pay"].astype("str")
fulldata["type_pay_special"] = fulldata["type_pay"].apply(lambda x:findname(x))
fulldata

d:\gongju\python3.6 64\anzhuang\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5,lossNUm,sts_order_number,type_pay_special
0,39.00,在线支付,2017-04-15 09:21:18,等待收货,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,2,2,0
1,39.00,在线支付,2017-04-15 09:21:18,完成,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,2,1,0
2,288.00,在线支付,2016-10-24 08:57:06,完成,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN,3,1,0
3,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,2,1,0
4,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,3,1,0
5,88.00,货到付款,2016-02-20 12:09:50,完成,137****8223,NaN,609460154345197832,7b14b3d3f23a46050cf6832c37354f84,eac9416f834d481d7c88da6134468d03,NaN,2,1,0
6,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,NaN,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,NaN,2,1,0
7,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,NaN,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,NaN,3,1,0
8,99.00,在线支付,NaN,完成,152****6998,NaN,452727175687180552,588c070f59bc7ca90301635d24c71b2f,fe94ed42923d9a010afd775f4fd1b472,NaN,4,1,0
9,16.80,在线支付,2016-12-07 02:57:57,完成,152****3139,NaN,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,NaN,2,1,0


In [6]:
mm = fulldata.ix[fulldata.sts_order_number == 1]
len(mm.id.unique())
# len(fulldata.id.unique())  #197960

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


162511

In [7]:
mm = fulldata.ix[fulldata.sts_order_number == 3]
len(mm.id.unique())

47617

In [5]:
#时间项很有问题
import time
def gettime7(x):  #修改时间格式
    x = str(x)
    tmp = x.split("-")
    if len(tmp) < 2:
        if len(x) > 10:
            x = x[0:10]
        x = int(x)
        time_local = time.localtime(x)
        return time.strftime("%Y-%m-%d %H:%M:%S",time_local)
    else:
        return x
def gettime8(x):
    t = time.strptime(x, '%Y-%m-%d %H:%M:%S')  #秒的话加上秒
    t = int(time.mktime(t))
    return t
def gettime9(x):
    x = str(x)
    tmp = x.split("-")[1]
    print(tmp)
import arrow
#arrow  时间函数，可以对时间进行统一编码

def parse_date(date_str, str_format='YYYY-MM-DD HH:mm:ss'):
    d = arrow.get(date_str, str_format)
    # 月初，月中，月末
    month_stage = int((d.day-1) / 10) + 1
    if month_stage == 4:
        month_stage = 3
    
    work_time = 0
    if (d.hour < 13 and d.hour > 8) or (d.hour < 18 and d.hour > 14):
        work_time = 1  # 1表示上班时间在购物  
    return (d.timestamp, d.year, d.month, d.day, d.week, d.isoweekday(), month_stage,work_time)    
def parse_time_order(date):
    d = parse_date(date, 'YYYY-MM-DD HH:mm:ss')
    return pd.Series(d, 
                  index=['orderInfo_timestamp', 'orderInfo_year', 'orderInfo_month',
                           'orderInfo_day', 'orderInfo_week', 'orderInfo_isoweekday',
                         'orderInfo_month_stage','orderInfo_work_time'], 
                  dtype=np.int32)
#将时间进行拆分，方便后续进行处理
# fulldata.time_order.value_counts()  
fulldata.time_order[fulldata[fulldata.time_order == "0"].index] = "2017-05-31 21:48:13"
# fulldata.time_order.value_counts().sort_values(ascending = False) 
fulldata["time_order"].fillna(fulldata["time_order"].value_counts().sort_values(ascending = False).index[0],inplace = True)
#拆分时间，月份  1——10，11——20，21——31    时间  上下午工作时间，非工作时间   
fulldata.time_order = fulldata.time_order.apply(lambda x:gettime7(x))

dateInfo = fulldata.time_order.apply(lambda x:parse_time_order(x))
fulldata = pd.concat([fulldata,dateInfo],axis = 1)
fulldata
#https://zhuanlan.zhihu.com/p/33063890  计算月消费能力

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5,...,sts_order_number,type_pay_special,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time
0,39.00,在线支付,2017-04-15 09:21:18,等待收货,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,...,1,0,1492248078,2017,4,15,15,6,2,1
1,39.00,在线支付,2017-04-15 09:21:18,完成,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,...,1,0,1492248078,2017,4,15,15,6,2,1
2,288.00,在线支付,2016-10-24 08:57:06,完成,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1477299426,2016,10,24,43,1,3,0
3,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1477598688,2016,10,27,43,4,3,0
4,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1477598688,2016,10,27,43,4,3,0
5,88.00,货到付款,2016-02-20 12:09:50,完成,137****8223,NaN,609460154345197832,7b14b3d3f23a46050cf6832c37354f84,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1455970190,2016,2,20,7,6,2,1
6,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,NaN,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1465818858,2016,6,13,24,1,2,1
7,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,NaN,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,NaN,...,1,0,1465818858,2016,6,13,24,1,2,1
8,99.00,在线支付,2017-05-31 21:48:13,完成,152****6998,NaN,452727175687180552,588c070f59bc7ca90301635d24c71b2f,fe94ed42923d9a010afd775f4fd1b472,NaN,...,1,0,1496267293,2017,5,31,22,3,3,0
9,16.80,在线支付,2016-12-07 02:57:57,完成,152****3139,NaN,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,NaN,...,1,0,1481079477,2016,12,7,49,3,1,0


In [6]:
len(fulldata.id.unique())

167960

In [7]:
fulldata.to_csv("../input/fulldata_order_undeal.csv",index = False)

# 下面是合并购买状态（实际没有做）

In [2]:
fulldata = pd.read_csv("../input/fulldata_order_undeal.csv")
del fulldata["unit_price"]
del fulldata["product_id_md5"]
fulldata.ix[fulldata.sts_order == 1].shape

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


(0, 19)

In [13]:
mm = fulldata.ix[fulldata.sts_order == 1]
len(mm.id.unique())   #有  5449 条ID属于中间状态

162519

In [15]:
cc = fulldata.ix[fulldata.sts_order == 2]
len(cc.id.unique()) 

11527

In [14]:
nn = fulldata.ix[fulldata.sts_order == 3]
len(nn.id.unique())

47433

In [16]:
dd = fulldata.ix[fulldata.sts_order == 4]
len(dd.id.unique())

2241

In [26]:
len(fulldata.id.unique())  #订单1和订单3 总数是 209952     162893

167968

In [27]:
state1 = fulldata.ix[fulldata.sts_order == 1]

In [28]:
state3 = fulldata.ix[fulldata.sts_order == 3]

In [10]:
fulldata.orderInfo_year.value_counts()

2016    1913289
2017     607867
2015     391337
2014     146248
2013      58016
2012      27960
2011       6540
2010       1147
2009        249
2008         74
2007         14
2006          4
Name: orderInfo_year, dtype: int64

In [37]:
fulldata.ix[fulldata.sts_order == 2]

,amt_order,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,lossNUm,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time
0,39.00,在线支付,2017-04-15 09:21:18,2,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,1492248078,2017,4,15,15,6,2,0
47,84.00,货到付款,2016-04-23 10:39:56,2,159****8718,587575870923739144,4b098cdfd1e0fe2d34f33b2d3840cdca,0bf08f99bd97a2ee522d94bbede23125,2,1461407996,2016,4,23,16,6,3,1
57,263.00,货到付款,2017-01-22 01:25:24,2,159****8718,587575870923739144,a61e101bb85789c5a0b55a4ffb3f66ee,0bf08f99bd97a2ee522d94bbede23125,2,1485048324,2017,1,22,3,7,3,0
775,88.00,在线支付,2017-02-12 08:41:39,2,135****9253,573245328367554568,987437b75c897f47d4304ba2f19437db,7d21e7e9d96a8b42803c3735da5b39c2,2,1486888899,2017,2,12,6,7,2,0
777,99.00,在线支付,2017-02-10 23:11:26,2,135****9253,573245328367554568,ef3862243bd55999a37caee0c00c227b,7d21e7e9d96a8b42803c3735da5b39c2,2,1486768286,2017,2,10,6,5,1,0
808,128.00,在线支付,2016-08-21 00:56:18,2,147****6940,584871736470671624,77a932d421a08744b46c4cd2b3c62b25,c42e375ac2ebe0536d83ec8fc25468bb,0,1471740978,2016,8,21,33,7,3,0
1061,238.00,货到付款,2016-02-25 13:25:40,2,159****0747,470154783844471048,2b26a088c42801cff128d433d8264b8d,a2046e5be6708d1e91a73d7cca5561a0,2,1456406740,2016,2,25,8,4,3,0
1104,149.00,在线支付,2017-03-14 19:52:13,2,139****6106,592035723775840520,db759f52d020b9b554c847847b5f2177,47b4867449631d7f4d60f0def432a588,2,1489521133,2017,3,14,11,2,2,0
1170,114.00,货到付款,2017-02-27 05:09:54,2,134****8982,584658324117524744,5a222cbc801634703b01082eaa7e215a,d5660633b8aad52cdee7f502eeabb759,2,1488172194,2017,2,27,9,1,3,0
1201,0.00,在线支付,2017-03-04 14:57:30,2,181****1727,575333856677204232,89be3b869d15d71867a20c1093e1a557,3621299a55ebb5c8a2acac21a00b2fd5,2,1488639450,2017,3,4,9,6,1,0


In [4]:
tmpdata = (pd.concat([fulldata.ix[fulldata.sts_order_number == 2], 
                      fulldata.ix[fulldata.sts_order_number == 1]],axis =0,ignore_index = False))
tmpdata
# tmpdata.ix[tmpdata.id == "609460154345197832"]

,amt_order,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,lossNUm,sts_order_number,type_pay_special,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time
0,39.00,在线支付,2017-04-15 09:21:18,等待收货,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,2,0,1492248078,2017,4,15,15,6,2,0
47,84.00,货到付款,2016-04-23 10:39:56,等待收货,159****8718,587575870923739144,4b098cdfd1e0fe2d34f33b2d3840cdca,0bf08f99bd97a2ee522d94bbede23125,2,2,0,1461407996,2016,4,23,16,6,3,1
57,263.00,货到付款,2017-01-22 01:25:24,等待收货,159****8718,587575870923739144,a61e101bb85789c5a0b55a4ffb3f66ee,0bf08f99bd97a2ee522d94bbede23125,2,2,0,1485048324,2017,1,22,3,7,3,0
775,88.00,在线支付,2017-02-12 08:41:39,等待收货,135****9253,573245328367554568,987437b75c897f47d4304ba2f19437db,7d21e7e9d96a8b42803c3735da5b39c2,2,2,0,1486888899,2017,2,12,6,7,2,0
777,99.00,在线支付,2017-02-10 23:11:26,等待收货,135****9253,573245328367554568,ef3862243bd55999a37caee0c00c227b,7d21e7e9d96a8b42803c3735da5b39c2,2,2,0,1486768286,2017,2,10,6,5,1,0
808,128.00,在线支付,2016-08-21 00:56:18,等待收货,147****6940,584871736470671624,77a932d421a08744b46c4cd2b3c62b25,c42e375ac2ebe0536d83ec8fc25468bb,0,2,0,1471740978,2016,8,21,33,7,3,0
1061,238.00,货到付款,2016-02-25 13:25:40,等待收货,159****0747,470154783844471048,2b26a088c42801cff128d433d8264b8d,a2046e5be6708d1e91a73d7cca5561a0,2,2,0,1456406740,2016,2,25,8,4,3,0
1104,149.00,在线支付,2017-03-14 19:52:13,等待收货,139****6106,592035723775840520,db759f52d020b9b554c847847b5f2177,47b4867449631d7f4d60f0def432a588,2,2,0,1489521133,2017,3,14,11,2,2,0
1170,114.00,货到付款,2017-02-27 05:09:54,等待收货,134****8982,584658324117524744,5a222cbc801634703b01082eaa7e215a,d5660633b8aad52cdee7f502eeabb759,2,2,0,1488172194,2017,2,27,9,1,3,0
1201,0.00,在线支付,2017-03-04 14:57:30,商品出库,181****1727,575333856677204232,89be3b869d15d71867a20c1093e1a557,3621299a55ebb5c8a2acac21a00b2fd5,2,2,0,1488639450,2017,3,4,9,6,1,0


In [ ]:
# tmp.to_csv("../input/tmp.csv",index = False)

# 下面是合并同一个订单号的记录，即一条记录一个订单号
    最后没有用合并的状态，仍然用状态1和2表示成功；3和4表示失败

In [25]:
# fulldata = pd.read_csv("../input/fulldata_order_undeal.csv") #fulldata_order_undeal.csv
# del fulldata["unit_price"]
# del fulldata["product_id_md5"]
# fulldata   "../input/fulldata_order_undeal111.csv"
fulldata = pd.read_csv("../input/fulldata_order_undeal111.csv")#, low_memory=False)
del fulldata["unit_price"]
del fulldata["product_id_md5"]
print(fulldata.isnull().sum())
fulldata  

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


amt_order                0
type_pay                 0
time_order               0
sts_order                0
phone                    0
id                       0
no_order_md5             0
name_rec_md5             0
lossNUm                  0
sts_order_number         0
type_pay_special         0
orderInfo_timestamp      0
orderInfo_year           0
orderInfo_month          0
orderInfo_day            0
orderInfo_week           0
orderInfo_isoweekday     0
orderInfo_month_stage    0
orderInfo_work_time      0
dtype: int64


,amt_order,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,lossNUm,sts_order_number,type_pay_special,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time
0,39.00,在线支付,2017-04-15 09:21:18,等待收货,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,2,0,1492248078,2017,4,15,15,6,2,1
1,39.00,在线支付,2017-04-15 09:21:18,完成,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,1,0,1492248078,2017,4,15,15,6,2,1
2,288.00,在线支付,2016-10-24 08:57:06,完成,137****8223,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,3,1,0,1477299426,2016,10,24,43,1,3,0
3,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,2,1,0,1477598688,2016,10,27,43,4,3,0
4,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,3,1,0,1477598688,2016,10,27,43,4,3,0
5,88.00,货到付款,2016-02-20 12:09:50,完成,137****8223,609460154345197832,7b14b3d3f23a46050cf6832c37354f84,eac9416f834d481d7c88da6134468d03,2,1,0,1455970190,2016,2,20,7,6,2,0
6,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,2,1,0,1465818858,2016,6,13,24,1,2,1
7,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,3,1,0,1465818858,2016,6,13,24,1,2,1
8,99.00,在线支付,2017-05-31 21:48:13,完成,152****6998,452727175687180552,588c070f59bc7ca90301635d24c71b2f,fe94ed42923d9a010afd775f4fd1b472,4,1,0,1496267293,2017,5,31,22,3,3,0
9,16.80,在线支付,2016-12-07 02:57:57,完成,152****3139,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,2,1,0,1481079477,2016,12,7,49,3,1,0


In [26]:
cc = fulldata.ix[fulldata.sts_order_number == 1]
# cc.shape,len(cc.id.unique())
cc

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,amt_order,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,lossNUm,sts_order_number,type_pay_special,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time
1,39.0,在线支付,2017-04-15 09:21:18,完成,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,1,0,1492248078,2017,4,15,15,6,2,1
2,288.0,在线支付,2016-10-24 08:57:06,完成,137****8223,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,3,1,0,1477299426,2016,10,24,43,1,3,0
3,428.0,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,2,1,0,1477598688,2016,10,27,43,4,3,0
4,428.0,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,3,1,0,1477598688,2016,10,27,43,4,3,0
5,88.0,货到付款,2016-02-20 12:09:50,完成,137****8223,609460154345197832,7b14b3d3f23a46050cf6832c37354f84,eac9416f834d481d7c88da6134468d03,2,1,0,1455970190,2016,2,20,7,6,2,0
6,259.0,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,2,1,0,1465818858,2016,6,13,24,1,2,1
7,259.0,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,3,1,0,1465818858,2016,6,13,24,1,2,1
8,99.0,在线支付,2017-05-31 21:48:13,完成,152****6998,452727175687180552,588c070f59bc7ca90301635d24c71b2f,fe94ed42923d9a010afd775f4fd1b472,4,1,0,1496267293,2017,5,31,22,3,3,0
9,16.8,在线支付,2016-12-07 02:57:57,完成,152****3139,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,2,1,0,1481079477,2016,12,7,49,3,1,0
10,16.8,在线支付,2016-12-07 02:57:57,完成,152****3139,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,2,1,0,1481079477,2016,12,7,49,3,1,0


In [27]:
def hebing(hebinggroup):
#     print("hebing................")
#     print(hebinggroup)
    hebinggroup.amt_order.iloc[0] = np.sum(hebinggroup.amt_order)
#     print("zz",hebinggroup)
    return hebinggroup.iloc[0,:]
def mygroup(group):
#     print(".................")
#     print(group)
    tmp = group.groupby(by = 'no_order_md5').apply(hebing)
    return tmp
# kk = cc.ix[cc.id == '521501199715799048']
# tmp = cc[0:3000].groupby(by = 'id').apply(mygroup)
tmp11 = cc.groupby(by = 'id').apply(mygroup)
# 609460154345197832   

In [28]:
tmp11.to_csv("../input/fulldata_oreder_undealhebingorder.csv",index = False)
tmp.ix[tmp.id == '521501199715799048'],cc.ix[cc.id == '521501199715799048']

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


(                                                     amt_order type_pay  \
 id                 no_order_md5                                           
 521501199715799048 0672d18129fc680da2b0e48a318b04b5       93.0     在线支付   
                    ec7fb7e582a2099e0c2e22d6f85f93a0       33.6     在线支付   
 
                                                               time_order  \
 id                 no_order_md5                                            
 521501199715799048 0672d18129fc680da2b0e48a318b04b5  2016-05-02 02:58:19   
                    ec7fb7e582a2099e0c2e22d6f85f93a0  2016-12-07 02:57:57   
 
                                                     sts_order        phone  \
 id                 no_order_md5                                              
 521501199715799048 0672d18129fc680da2b0e48a318b04b5        完成  187****6953   
                    ec7fb7e582a2099e0c2e22d6f85f93a0        完成  152****3139   
 
                                                                  

In [31]:
tmp11['sts_order_number'].value_counts()

1    2289074
Name: sts_order_number, dtype: int64

In [29]:
len(tmp11.id.unique())

162519

In [6]:
 521501199715799048  ec7fb7e582a2099e0c2e22d6f85f93a0   
10  521501199715799048  ec7fb7e582a2099e0c2e22d6f85f93a0   
11  521501199715799048  0672d18129fc680da2b0e48a318b04b5   
12  521501199715799048  0672d18129fc680da2b0e48a318b04b5   

fulldata.to_csv("../input/fulldata_oreder_undeal2222.csv",index = False)

In [5]:
fulldata = pd.read_csv("../input/fulldata_oreder_undeal2222.csv")
print(fulldata.columns)
fulldata

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['amt_order', 'type_pay', 'time_order', 'sts_order', 'phone', 'id',
       'no_order_md5', 'name_rec_md5', 'lossNUm', 'sts_order_number',
       'type_pay_special', 'orderInfo_timestamp', 'orderInfo_year',
       'orderInfo_month', 'orderInfo_day', 'orderInfo_week',
       'orderInfo_isoweekday', 'orderInfo_month_stage', 'orderInfo_work_time',
       'whitePayment'],
      dtype='object')


,amt_order,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,lossNUm,sts_order_number,type_pay_special,orderInfo_timestamp,orderInfo_year,orderInfo_month,orderInfo_day,orderInfo_week,orderInfo_isoweekday,orderInfo_month_stage,orderInfo_work_time,whitePayment
0,39.00,在线支付,2017-04-15 09:21:18,等待收货,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,1,0,1492248078,2017,4,15,15,6,2,0,0
1,39.00,在线支付,2017-04-15 09:21:18,完成,159****2937,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,2,1,0,1492248078,2017,4,15,15,6,2,0,0
2,288.00,在线支付,2016-10-24 08:57:06,完成,137****8223,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,3,1,0,1477299426,2016,10,24,43,1,3,0,0
3,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,2,1,0,1477598688,2016,10,27,43,4,3,0,0
4,428.00,在线支付,2016-10-27 20:04:48,完成,137****8223,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,3,1,0,1477598688,2016,10,27,43,4,3,0,0
5,88.00,货到付款,2016-02-20 12:09:50,完成,137****8223,609460154345197832,7b14b3d3f23a46050cf6832c37354f84,eac9416f834d481d7c88da6134468d03,2,1,0,1455970190,2016,2,20,7,6,2,0,0
6,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,2,1,0,1465818858,2016,6,13,24,1,2,1,0
7,259.00,货到付款,2016-06-13 11:54:18,完成,137****8223,609460154345197832,f2b76d2f4ada342fdd4a83f06835756e,eac9416f834d481d7c88da6134468d03,3,1,0,1465818858,2016,6,13,24,1,2,1,0
8,99.00,在线支付,2017-05-31 21:48:13,完成,152****6998,452727175687180552,588c070f59bc7ca90301635d24c71b2f,fe94ed42923d9a010afd775f4fd1b472,4,1,0,1496267293,2017,5,31,22,3,4,0,0
9,16.80,在线支付,2016-12-07 02:57:57,完成,152****3139,521501199715799048,ec7fb7e582a2099e0c2e22d6f85f93a0,ccbc99481f4ad7795fb19cb0039831b7,2,1,0,1481079477,2016,12,7,49,3,1,0,0


In [7]:
fulldata.sts_order_number.value_counts()

1    2706682
3     446063
Name: sts_order_number, dtype: int64

# 下面开始统计相关信息

In [3]:
cc = 0
num = 167960
bili = 0.1
def mygroupCount(group11):
    group = group11.ix[group11.sts_order_number == 1]

    global cc,num,bili
    cc += 1 
    tmp = np.round(cc/num,1)
    if tmp == bili:
        bili += 0.1
        print("已完成:",tmp*100,"%")
        
    if group.shape[0] == 0:
#         print("全是退款取消的信息")
        group = group11.ix[group11.sts_order_number == 3]
    count = len(group.no_order_md5.unique())   #    将一个订单的内容合并起来
    
    #统计这个人是否抢中什么，看待运气的心里
    orderInfo_luck = 0
    tmp = group.ix[group.sts_order == "未抢中"]
    if tmp.shape[0] > 0:
        orderInfo_luck = tmp.shape[0]/count

    #有几个收货人
    orderInfo_name_rec_md5_num = len(group.name_rec_md5.unique())
    
    #统计是否经常在 上班时间购物
    orderInfo_work_time_more_cate = (group.orderInfo_work_time.value_counts()
                                     .sort_values(ascending = False).index[0])
    
    orderInfo_work_time_more_proportion = (group.orderInfo_work_time.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
           
    # 统计在每年的 多少周购物比较多 orderInfo_week
    orderInfo_week_more_cate = (group.orderInfo_week.value_counts()
                                     .sort_values(ascending = False).index[0])

    tmp = group.orderInfo_week.value_counts().sort_values(ascending = False).iloc[0]
    orderInfo_week_more_proportion = (tmp/count)
    #这里时间必须 用.iloc[0]  也是神奇，以后还是都带上这个，免得找不到错误在哪里
    
    # 哪一年,月，日，周几购物多
    orderInfo_year_more_cate = (group.orderInfo_year.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_year_more_proportion = (group.orderInfo_year.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
    
    orderInfo_month_more_cate = (group.orderInfo_month.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_month_more_proportion = (group.orderInfo_month.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
           
    orderInfo_day_more_cate = (group.orderInfo_day.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_day_more_proportion = (group.orderInfo_day.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
    
    orderInfo_isoweekday_more_cate = (group.orderInfo_isoweekday.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_isoweekday_more_proportion = (group.orderInfo_isoweekday.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
    
    orderInfo_month_stage_more_cate = (group.orderInfo_month_stage.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_month_stage_more_proportion = (group.orderInfo_month_stage.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
     
    #有多少种支付方式
    orderInfo_type_pay_cate_num = len(group.type_pay.unique())
    #常见支付方式及其比率
    orderInfo_type_pay_more_cate = (group.type_pay.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_type_pay_more_proportion = (group.type_pay.value_counts()
                                     .sort_values(ascending = False).iloc[0]/count)
    
    #使用京豆京劵等支付的比例（说明比较活跃）  type_pay_special   1 
    tmp = group.ix[group.type_pay_special == 1]
    orderInfo_type_pay_special_proportion = tmp.shape[0]/count

    #使用白条、分期的比例是多少
    tmp = group.ix[group.whitePayment == 1]
    orderInfo_whitePayment_proportion = tmp.shape[0]/count
   
    #有多少种订单状态
    orderInfo_sts_order_cate_num = len(group.sts_order.unique())   
    #常见订单状态及其比率
    orderInfo_sts_order_more_cate = (group.sts_order.value_counts()
                                     .sort_values(ascending = False).index[0])
    orderInfo_sts_order_more_proportion = (group.sts_order.value_counts()
                                     .sort_values(ascending = False)[0]/count)
    
     #有多少手机号
    orderInfo_phone_cate_num = len(group.phone.unique())
    
    #花费充值单次最大金额，平均每次充值多少,充话费的比例
    tmp = group.ix[group.sts_order == "充值成功"]
    orderInfo_phone_money_max = 0  #有的人没有充值花费
    orderInfo_phone_money_ave = 0
    if tmp.shape[0] > 0:
        orderInfo_phone_money_max = np.max(tmp.amt_order)
        orderInfo_phone_money_ave = np.mean(tmp.amt_order)
    orderInfo_phone_money_proportion = tmp.shape[0]/count
    
    #平均多久购物一次
    tmpA = np.max(group.orderInfo_timestamp)
    tmpB = np.min(group.orderInfo_timestamp)  
    orderInfo_buy_time_interval = (tmpA - tmpB)/count  

    #平均每次购物多少钱，购物金额最大的一笔
    orderInfo_buy_money_max = np.max(group.amt_order)
    orderInfo_buy_money_ave = (group.amt_order.sum())/count
    
    
    indexs = {
            "orderInfo_work_time_more_cate":orderInfo_work_time_more_cate,  
            "orderInfo_week_more_cate":orderInfo_week_more_cate,   
            "orderInfo_year_more_cate":orderInfo_year_more_cate,       
            "orderInfo_month_more_cate":orderInfo_month_more_cate,   
            "orderInfo_day_more_cate":orderInfo_day_more_cate,   
            "orderInfo_isoweekday_more_cate":orderInfo_isoweekday_more_cate,      
            "orderInfo_month_stage_more_cate":orderInfo_month_stage_more_cate,  
            "orderInfo_type_pay_more_cate": orderInfo_type_pay_more_cate, 
            "orderInfo_sts_order_more_cate": orderInfo_sts_order_more_cate, 

            "orderInfo_work_time_more_proportion":orderInfo_work_time_more_proportion, 
            "orderInfo_luck":orderInfo_luck,
            "orderInfo_name_rec_md5_num":orderInfo_name_rec_md5_num, 
            "orderInfo_week_more_proportion": orderInfo_week_more_proportion, 
            "orderInfo_year_more_proportion": orderInfo_year_more_proportion, 
            "orderInfo_month_more_proportion": orderInfo_month_more_proportion, 
            "orderInfo_day_more_proportion": orderInfo_day_more_proportion, 
            "orderInfo_isoweekday_more_proportion": orderInfo_isoweekday_more_proportion, 
            "orderInfo_month_stage_more_proportion": orderInfo_month_stage_more_proportion, 
            "orderInfo_type_pay_cate_num":orderInfo_type_pay_cate_num,   
            "orderInfo_type_pay_more_proportion": orderInfo_type_pay_more_proportion, 
            "orderInfo_type_pay_special_proportion":orderInfo_type_pay_special_proportion,   
            "orderInfo_whitePayment_proportion": orderInfo_whitePayment_proportion, 
            "orderInfo_sts_order_cate_num":orderInfo_sts_order_cate_num,   
            "orderInfo_sts_order_more_proportion": orderInfo_sts_order_more_proportion, 
            "orderInfo_phone_cate_num":orderInfo_phone_cate_num,        
            "orderInfo_phone_money_max": orderInfo_phone_money_max,     
            "orderInfo_phone_money_ave":orderInfo_phone_money_ave,   
            "orderInfo_phone_money_proportion": orderInfo_phone_money_proportion, 
            "orderInfo_buy_time_interval": orderInfo_buy_time_interval,     
            "orderInfo_buy_money_max":orderInfo_buy_money_max,   
            "orderInfo_buy_money_ave": orderInfo_buy_money_ave
    }
    
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])
# tmp122 = fulldata.iloc[0:30000,:].groupby(by=['id']).apply(mygroupCount)
tmp122 = fulldata.groupby(by=['id']).apply(mygroupCount)
tmp122.insert(0,"id",tmp122.index)

d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


已完成: 10.0 %
已完成: 20.0 %


In [4]:
tmp122.to_csv("../dealinput/orderCount.csv",index = False)

In [5]:
tmp122

,id,orderInfo_work_time_more_cate,orderInfo_week_more_cate,orderInfo_year_more_cate,orderInfo_month_more_cate,orderInfo_day_more_cate,orderInfo_isoweekday_more_cate,orderInfo_month_stage_more_cate,orderInfo_type_pay_more_cate,orderInfo_sts_order_more_cate,...,orderInfo_whitePayment_proportion,orderInfo_sts_order_cate_num,orderInfo_sts_order_more_proportion,orderInfo_phone_cate_num,orderInfo_phone_money_max,orderInfo_phone_money_ave,orderInfo_phone_money_proportion,orderInfo_buy_time_interval,orderInfo_buy_money_max,orderInfo_buy_money_ave
id,,,,,,,,,,,,,,,,,,,,,
616266486771618056,616266486771618056,0,22,2017,5,31,3,4,在线支付,完成,...,0.00000,1,1.000000,1,0.0,0.000000,0.000000,0.000000e+00,99.0,99.000000
616411495579783176,616411495579783176,0,31,2016,8,15,4,1,在线支付,完成,...,0.00000,4,0.610390,8,199.6,70.909474,0.246753,4.172764e+05,5788.0,194.334026
616483294195749128,616483294195749128,0,21,2016,5,24,2,3,在线支付,完成,...,0.00000,3,0.714286,2,0.0,0.000000,0.000000,1.052259e+06,1399.0,352.475714
616547037910929672,616547037910929672,0,46,2016,11,22,2,3,在线支付,完成,...,0.00000,2,0.857143,1,0.0,0.000000,0.000000,4.977219e+05,148.0,84.657143
616550964970656008,616550964970656008,0,28,2016,3,23,3,2,在线支付,完成,...,0.00000,4,0.670886,9,99.8,43.916667,0.379747,4.604425e+05,4799.0,303.359494
616657485251481864,616657485251481864,0,27,2016,7,8,1,1,在线支付,完成,...,0.00000,2,0.581818,7,99.9,40.165870,0.418182,3.495712e+05,4401.0,159.763636
616758963400085768,616758963400085768,0,35,2016,8,30,2,3,在线支付,完成,...,0.00000,1,1.000000,1,0.0,0.000000,0.000000,0.000000e+00,32.2,32.200000
616772308744278024,616772308744278024,0,22,2017,5,31,3,4,在线支付,完成,...,0.00000,1,1.000000,1,0.0,0.000000,0.000000,0.000000e+00,99.0,99.000000
616777971096621320,616777971096621320,0,22,2017,5,31,3,4,在线支付,完成,...,0.00000,1,1.000000,1,0.0,0.000000,0.000000,0.000000e+00,99.0,99.000000
